In [1]:
import numpy as np

with open("testcases/5x5/input_5x5.txt", "r") as f:
    data = []
    for line in f:
        row = [int(x) for x in line.strip().split(",")]
        data.append(row)
    Input = np.array(data)

In [2]:
print(Input)

[[3 0 0 0 1]
 [0 0 2 0 0]
 [0 0 0 0 1]
 [3 0 0 0 2]
 [0 0 0 0 0]]


In [3]:
from itertools import combinations
from pysat.solvers import Glucose3


def solve_minesweeper(grid):
    n = len(grid)
    m = len(grid[0])
    solver = Glucose3()

    # Add clauses for each cell
    for i in range(n):
        for j in range(m):
            if grid[i][j] != 0:
                neighbors = []
                for x in range(max(0, i - 1), min(n, i + 2)):
                    for y in range(max(0, j - 1), min(m, j + 2)):
                        if x != i or y != j:
                            neighbors.append(x * m + y + 1)
                # Encode atmost constraint as CNF clauses
                for c in combinations(neighbors, grid[i][j] + 1):
                    solver.add_clause([-x for x in c])
                # Encode atleast constraint as CNF clauses
                for c in combinations(neighbors, len(neighbors) - grid[i][j] + 1):
                    solver.add_clause([x for x in c])

    # Check if the problem is solvable
    if not solver.solve():
        return None

    # Extract solution
    model = solver.get_model()

    # Debugging
    # print(f"n: {n}, m: {m}, len(model): {len(model)}")
    # print(model)
    solution = [[0] * m for _ in range(n)]
    for i in range(n):
        for j in range(m):
            try:
                if model[(i * m + j)] > 0:
                    solution[i][j] = 1
            except IndexError:
                pass
    return solution

In [4]:
# Mine location
solution = solve_minesweeper(Input)
print(np.array(solution))

[[0 1 0 0 0]
 [1 1 0 0 1]
 [1 0 0 0 1]
 [0 1 0 0 0]
 [1 0 0 1 0]]


In [5]:
# Assign mine location = -1
def output(Input, solution):
    for i in range(len(Input)):
        for j in range(len(Input[0])):
            if solution[i][j] == 1:
                Input[i][j] = -1
    return Input

In [6]:
def show_Matrix(A):
    for i in range(len(A)):
        for j in range(len(A[0])):
            print("%s " % A[i][j], end="")
        print()
    print()

In [7]:
result = output(Input, solution)

In [8]:
output = [["X" if x == -1 else x for x in row] for row in result]
show_Matrix(output)
np.savetxt("output_20x20.txt", output, delimiter=",", fmt="%s")

3 X 0 0 1 
X X 2 0 X 
X 0 0 0 X 
3 X 0 0 2 
X 0 0 X 0 

